# Домашнее задание 4
# Свиридов Иван СКБ182
# Проверка статистических гипотез

### Импортируем модули

In [53]:
from math import sqrt
from IPython.display import Image
import numpy as np
import random
import matplotlib.pyplot as pyplot
import statsmodels.api
from statsmodels.distributions.empirical_distribution import ECDF
from collections import Counter

## Экспоненциальное распределение 


Экспоненциальное распределение является непрерывным, как мы знаем из первой домашней работы. А его функция распределения тоже непрерывна. Из этого следует, что мы можем применить критерий Колмогорова, так как он применяется в тех случаях, когда функция $F(x)$ непрерывна. Тогда рассмотрим простые и сложные гипотезы. И следующим алгоритмом будем пользоваться 

### Генерируем выборки для экспоненциального распределения
Сгенерируем выборки размера n=1000

In [39]:
def generate(N=8,n=5):
    def inverse_function(eta, lamb=1):
        return -1/lamb * np.log(1-eta)
    eta = random.random()
    samples = []
    for _ in range(n):
        eta = random.random()
        modified_eta = inverse_function(eta)
        samples.append(modified_eta)
    return samples
generate()
    
def gen_samples():
    volume_list = [5,10,100,1000]
    var_list = []
    for v in volume_list:
        for i in range(5):
            var_list.append(generate(n=v))

    return(var_list)
var_list = gen_samples()


### Критерий Колмогорова для экспоненциального распределения
### Простая гипотеза
$H_{0}:\xi~exp(\lambda=1)=F_{\xi}(x)=F(x)$


Сгенерируем эмпирическую функцию распределения. 

In [53]:
x_list=var_list[1]
ECDF=statsmodels.distributions.empirical_distribution.ECDF(x_list, side='right')
print("X are: ", ECDF.x )
print("Y are: ",ECDF.y)

X are:  [          -inf 6.86774553e-04 1.51121188e-03 ... 5.58352386e+00
 5.91113262e+00 6.02886084e+00]
Y are:  [0.    0.001 0.002 ... 0.998 0.999 1.   ]


Сгенерируем теоретическую функцию распределения

In [54]:
CDF = [1-np.exp(-i) for i in ECDF.x[1:]]
CDF.insert(0,0)

Статистика критерия определеяется формулой:
$$D_{n}=sup_{x \in R}|\hat{F}_{n}(x)-F(x)|$$
Представляет  собой макисмальное отклонение эмпирической функции $\hat{F}_{n}(x)$, построенной по выборке  $X$, от гипотетической функции распределения $F(x)$. $\hat{F}_{n}(x)$ является состоятельной и оптимальной оценкой для $F(x)$. При больших $n$ мы будем наблюдать сближение $\hat{F}_{n}(x)$ и $F(x)$. то есть их разность (а у нас это $D_{n}$) будет стремится стремится к нулю.

In [55]:
D=[]
for i in range(1, len(CDF)):
    D.append(abs(ECDF.y[i]-CDF[i]))
print("D_n = ", max(D))


D_n =  0.014350102783714624


Критическая граница $t_{\alpha}$ при заданном уровне значимости $\alpha$ рассчитвается при этом на основании теоремы Колмогорова. А именно, если $n$ достаточно велико $(n \geq 20)$, то положив $t_{\alpha}=\tfrac{\lambda_{\alpha}}{\sqrt{n}}$
$$P(\sqrt{n}D_{n}\geq\sqrt{n}t_{\alpha}|H_{0})\approx 1 - K(t_{\alpha})=\alpha$$

Возьмем уровень значимости $\alpha=0,005$. Тогда по таблицу найдем $\lambda_{\alpha}$. И тогда $\lambda_{\alpha}=1,36$. Мы брали выборку объема $n=1000$, тогда вычислим $t_{\alpha}=\tfrac{\lambda_{\alpha}}{\sqrt{n}}$

In [56]:
talpha= 1.36/sqrt(len(var_list[1]))
print("t=", talpha)


t= 0.04300697617828996


Выше мы вычислили $D_{n}=$ и $t_{\alpha}=0,043...$.

Тогда проверим гипотезу. Если $D_{n}< t_{\alpha}$, тогда гипотеза принимается. Если $D_{n}\geq t_{\alpha}$, тогда гипотеза отвергается.
Проверим это:

In [57]:
if max(D)<talpha:
    print('Гипотеза принимается')
else:
    print('Гипотеза не принимается')

Гипотеза принимается


Как мы получиливыяснили, наша гипотеза принимается

### Сложная гипотеза

Для проверки сложной гипотезы, волспользуемся определенным методом.Нам придется составить две выборки. И по одной выборке провести оценку параметра $\theta$, а по другой выборке мы проверим как раз таки гипотезу по критерию Колмогорова. 

Это будет выборка для оценки параметра $\theta$

In [58]:
print(var_list[1])

[0.41887258530362753, 1.1569111667375662, 0.18627732492099522, 0.5089423385792687, 0.30074689632670565, 1.4548105994457865, 2.6198665377778996, 0.2719000676993434, 0.008971311590821485, 4.09676723037647, 1.4987316382409877, 0.7418512906072005, 0.044181269587094577, 0.20226633109510064, 0.785263914918073, 0.12597988621707573, 0.5652853818753978, 0.08558972500408217, 0.7759297769612439, 0.07822606695177356, 0.020703280523109307, 3.7611424571896643, 0.7529887667448608, 0.7571953914748342, 0.08658132191818164, 3.9017285453586226, 1.8432371147190862, 0.7417402116836086, 0.02938686997055829, 0.8351572782615125, 1.4058094425195398, 0.6833860676735166, 0.037884537740205344, 0.5788948190997222, 0.43025510473137346, 2.4282166560927765, 1.701892646739132, 2.5785807518386026, 0.29365607075190525, 1.6941910117470103, 0.9222940011931621, 0.26969707561776163, 0.37433816223801586, 0.5532449190354188, 0.5057424991920051, 0.9001894588306826, 1.8135826440396512, 1.3909622997083566, 0.8257626935447703, 0.

Найдем оценку параметра $\theta$ 

In [59]:
a = len(var_list[1])#количество элементов 
b = sum(var_list[1])#сумма всех элементов

In [78]:
theta = 1/(b/a)
print(theta)

0.9786155505386381


Мы получили оценку. А тепер
А дальше делаем по аналогии с простой гипотезой, но уже для другой (второй) выборки.

Сгенерируем эмпирическую функцию распределения

In [79]:
x_list=var_list[2]
ECDF=statsmodels.distributions.empirical_distribution.ECDF(x_list, side='right')
print("X are: ", ECDF.x )
print("Y are: ",ECDF.y)

X are:  [          -inf 3.08049625e-04 2.13746527e-03 ... 7.52463105e+00
 8.21125491e+00 9.03917888e+00]
Y are:  [0.    0.001 0.002 ... 0.998 0.999 1.   ]


Сгенерируем теоретическую функцию

In [80]:
CDF = [1-np.exp(-i*theta) for i in ECDF.x[1:]]
CDF.insert(0,0)

Статистика критерия определеяется формулой:
$$D_{n}=sup_{x \in R}|\hat{F}_{n}(x)-F(x)|$$
Представляет  собой макисмальное отклонение эмпирической функции $\hat{F}_{n}(x)$, построенной по выборке  $X$, от гипотетической функции распределения $F(x)$. $\hat{F}_{n}(x)$ является состоятельной и оптимальной оценкой для $F(x)$. При больших $n$ мы будем наблюдать сближение $\hat{F}_{n}(x)$ и $F(x)$. то есть их разность (а у нас это $D_{n}$) будет стремится стремится к нулю.

In [81]:
D=[]
for i in range(1, len(CDF)):
    D.append(abs(ECDF.y[i]-CDF[i]))
print("sup = ", max(D))

sup =  0.03131215608854204


Критическая граница $t_{\alpha}$ при заданном уровне значимости $\alpha$ рассчитвается при этом на основании теоремы Колмогорова. А именно, если $n$ достаточно велико $(n \geq 20)$, то положив $t_{\alpha}=\tfrac{\lambda_{\alpha}}{\sqrt{n}}$
$$P(\sqrt{n}D_{n}\geq\sqrt{n}t_{\alpha}|H_{0})\approx 1 - K(t_{\alpha})=\alpha$$

Возьмем уровень значимости $\alpha=0,005$. Тогда по таблицу найдем $\lambda_{\alpha}$. И тогда $\lambda_{\alpha}=1,36$. Мы брали выборку объема $n=1000$, тогда вычислим $t_{\alpha}=\tfrac{\lambda_{\alpha}}{\sqrt{n}}$

In [82]:
talpha= 1.36/sqrt(len(var_list[1]))
print("t=", talpha)

t= 0.04300697617828996


Выше мы вычислили $D_{n}=$ и $t_{\alpha}=0,043...$.

Тогда проверим гипотезу. Если $D_{n}< t_{\alpha}$, тогда гипотеза принимается. Если $D_{n}\geq t_{\alpha}$, тогда гипотеза отвергается.
Проверим это:

In [83]:
if max(D)<talpha:
    print('Гипотеза принимается')
else:
    print('Гипотеза не принимается')

Гипотеза принимается


## Дискретное равномерное распределение 

Дискретное равномерное распределение не является непрерывным, как мы выясниил в первом домашнем задании. Из этого можно сделать вывод, что сы не сможем применить критерий Колмогорова для простой и сложной гипотез для данного распределения. 

### Генерируем выборки для дискретного равномерного распределения
Сгенерируем выборку объема n=1000

In [50]:
def generate(N=16,n=1):
    lx=[]
    dic = {}
    for i in range(N):
        dic[i+1]=(i/N,(i+1)/N)
    for i in range(n):
        rand_number = random.random()
        for i in dic:
            if dic[i][0] <= rand_number < dic[i][1]:
                x = i
        lx.append(x)
    return(lx)

generate()

def gen_samples():
    volume_list = [1000]
    samples_list = []
    for v in volume_list:
        for i in range(5):
            samples_list.append(generate(n=v))
    return(samples_list)
samples_list = gen_samples()


In [49]:
first_sample = samples_list[1]

In [51]:
second_sample = samples_list[2]

### Критерий $X^{2}$ для дискретного равномерного распределения
### Простая гипотеза

Выдвенем гипотезу, о том что является ли на наше неизвестное распределение, равномерно дискретным. Мы будем рассматривать простую гипотезу, поэтому у нас будет известен параметр $\theta=8$. Запишем в красивую форму.
$H_{0}=[\xi \sim R_{[1,8]}]=[F_{\xi}(x)-F(x)]$


Первым делом, необходимо по нашей выборке посчитать частоты $v_{j}$

Так в нашей выборке, элементы могут принимать только натуральные значения, тогда мы будем считать частоты для чисел: 1,2,3,4,5,6,7,8

Воспользуемся формулой для нахождения частот:
$$v_{j}=\sum\limits_{i=1}^{n} I(\xi_{i}=j), \text{ где } j=1,..,8$$

Посчитаем частоты нашей выборки:

In [68]:
chastot=dict(Counter(samples_list[1]))
print(chastot)

{3: 119, 2: 126, 6: 126, 7: 133, 1: 117, 8: 114, 5: 142, 4: 123}


Преобразуем в более удобный вид.

In [86]:
chastota = [117, 126, 119, 123, 142, 126, 133, 114]

Таким образом мы получили вектор частот, который имеет полиноминальное распределение.

Составим вектор вероятностей. Т.к. Функция вероятности для дискретного равномерного распределения имеет вид $\tfrac{1}{n}$, где $n=b-a+1=8-1+1=8$

In [87]:
vecver=[1/8]*8
print(vecver)

[0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]


Нам необходимо проверить гипотезу $H_{0}$ на соответствие о том, что $p^{о}$-значение нашей предполагаемой вероятности. Как мы показали выше, оно будет равно $\tfrac{1}{8}$, равно $p$. ДЛя этого, воспользуемся $X^{2}$ и его формулой:
$$\dot{X}_{n}^{2}=\dot{X}_{n}^{2}(\nu)=\sum\limits_{j=1}^{n}\tfrac{(\nu_{j}-np_{j}^{0})^{2}}{np^{0}_{j}}$$

In [88]:
sum_xu_kv = 0
for i in range(8):
    sum_xu_kv = sum_xu_kv+((chastota[i]-nvar*vecver[i])**2)/(nvar*vecver[i])
print(sum_xu_kv)

4.640000000000001


Посколькку относительная частота $\tfrac{\nu_{j}}{n}$ события {$\xi=j$} является состоятельной оценкой его вероятности $p_{j}^{o} (j=1,...,n)$, при больших $n$ разности $|\tfrac{\nu_{j}}{n}-p_{j}^{o}|$ должны быть малы, следовательно, и значение статистики $X^{2}_{n}$ не должно быть слишком большим. Поэтому естественно задать критическую область для гипотезы $H_{0}$ в виде $T=${$\dot{X}_{n}^{2}>t_{\alpha}$}, где критическая граница $t_{\alpha}$ при заданном уровне значимости $\alpha$ должна быть выбрана из условия 
$$P[\dot{X}_{n}^{2}>t_{\alpha}|H_{0}]=\alpha$$
$$P[\dot{X}_{n}^{2}>t_{\alpha}|H_{0}]\approx 1-F_{N-1}(t_{\alpha})\to t_{\alpha}=X_{1-\alpha, N-1}^{2}$$

У нас дан уровень значимости $\alpha=0,05$ и N=8-1=7. Тогда по таблице найдем $t_\alpha$, который будет равен 14,1 .

Тогда проверим гипотезу. 
Мы нашли значение $\dot{X}_{n}^{2}(\nu)=4.64...$ и $t_{\alpha}=14,1$

In [90]:
talpha=14.1
if sum_xu_kv<talpha:
    print('Гипотеза принимается')
else:
    print('Гипотеза не принимается')

Гипотеза принимается


### Сложная гипотеза

Выдвенем гипотезу $H_{0}:F(x)=F_{\theta}(x)$

У нас имеется неизвестный параметр $\theta$. И нам необходимо получить его оценку. Тогда, для критерия Пирсона возьмем оценку методом максимального правдоподобия. Тогда:
$$L(x|\theta)=\theta^{-n} Ind(X_{1}>1)*Ind(X_{n}\geq \theta)\to \hat{\theta}=X_{n}$$

Вычислим $X_n$ для нашей выборки

In [93]:
X_n=max(samples_list[1])
print(X_n)

8


Таким образом, мы получили $X_{n}=8$

В дальнейшем алгоритм схож с алгоритмом для простой гипотезы.

Необходимо по нашей выборке посчитать частоты $v_{j}$

Так в нашей выборке, элементы могут принимать только натуральные значения, тогда мы будем считать частоты для чисел: 1,2,3,4,5,6,7,8

Воспользуемся формулой для нахождения частот:
$$v_{j}=\sum\limits_{i=1}^{n} I(\xi_{i}=j), \text{ где } j=1,..,8$$

Посчитаем частоты нашей выборке:

In [94]:
chastot=dict(Counter(samples_list[1]))
print(chastot)

{3: 119, 2: 126, 6: 126, 7: 133, 1: 117, 8: 114, 5: 142, 4: 123}


Преобразуем в более нормальный вид:

In [95]:
chastota =[117, 126, 119, 123, 142, 126, 133, 114]

Таким образом, мы получили вектор частот $\nu$

Так как мы рассматривает сложную гипотезу, тогда вектор вероятностей будет зависить от параметра $\theta$, то есть $p^{o}=p^{o}(\theta)$

Как мы показали вверху $\theta=X_{n}=8$

Тогда составим вектор вероятностей:

In [96]:
vecver=[1/X_n]*8
print(vecver)

[0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]


In [97]:
sum_xu_kv = 0
for i in range(8):
    sum_xu_kv = sum_xu_kv+((chastota[i]-nvar*vecver[i])**2)/(nvar*vecver[i])
print(sum_xu_kv)

4.640000000000001


Посколькку относительная частота $\tfrac{\nu_{j}}{n}$ события {$\xi=j$} является состоятельной оценкой его вероятности $p_{j}^{o} (j=1,...,n)$, при больших $n$ разности $|\tfrac{\nu_{j}}{n}-p_{j}^{o}|$ должны быть малы, следовательно, и значение статистики $X^{2}_{n}$ не должно быть слишком большим. Поэтому естественно задать критическую область для гипотезы $H_{0}$ в виде $T=${$\dot{X}_{n}^{2}>t_{\alpha}$}, где критическая граница $t_{\alpha}$ при заданном уровне значимости $\alpha$ должна быть выбрана из условия 
$$P[\dot{X}_{n}^{2}>t_{\alpha}|H_{0}]=\alpha$$
$$P[\dot{X}_{n}^{2}>t_{\alpha}|H_{0}]\approx 1-F_{N-1}(t_{\alpha})\to t_{\alpha}=X_{1-\alpha, N-1}^{2}$$

У нас дан уровень значимости $\alpha=0,05$ и N=8-1=7. Тогда по таблице найдем $t_\alpha$, который будет равен 14,1 .

Тогда проверим гипотезу. 
Мы нашли значение $\dot{X}_{n}^{2}(\nu)=4.64...$ и $t_{\alpha}=14,1$

In [43]:
talpha=14.1
if sum_xu_kv<talpha:
 print('Гипотеза принимается')
else:
 print('Гипотеза не принимается')

Гипотеза принимается


## Проверка гипотезы однородности

## Экспоненциальное распределение

Критерий однородности Смирнова применяется в задачах о двух выборках из непрерывного распределения. И его тестовая статистика будет иметь следующий вид:
$$D_{n,m}=sup|\hat{F}_{1n}(x)-\hat{F}_{2m}(x)|$$
$\hat{F}_{1n}(x), \hat{F}_{2m}(x)$ являются эмпирическими функциями распределения. Статистика $D_{n}$ не должна быть большой, так как эмпирическая функция распределения является оптимальной оценкой для теоретической и с увеличением выборки они сближаются, и если гипотезы верна, то они будут соединяться, сходится к теоретической функции распределения

Так как данное распределение является непрерывным, тогда мы можем воспользоваться критерием однородности Смирнова. Для применения данного критерия ,нам необходимо сгенерировать две выборки. 


Также, гипотеза $H_{0}$ отклоняется тогда и только тогда когда $D_{n,m}>t_{\alpha}\cdot \sqrt{\tfrac{1}{n}+\tfrac{1}{m}}$

При увеличении объема выборки, отклонение эмпирической функции распределения будет уменьшаться. 

## Дискретное равномерное распределение

Так как вмы рассматриваем дискретное равномерное распределение, тогда воспользуемся критерием однородности $X^{2}$. Воспользуемся выборками, которые мы генерировали выше. 

Посчитаем в каждой выборке число повторений элементов. То есть найдем частоты наших исходов. И воспользуемся формулой: $$\nu_{k}^{n}=\sum\limits_{j=1}^{n}Ind(\xi=j)$$

In [9]:
chastot_1 = dict(Counter(samples_list[1]))
chastot_2 = dict(Counter(samples_list[2]))
print(chastot_1)
print(chastot_2)

{6: 128, 8: 124, 2: 110, 4: 122, 1: 129, 3: 115, 7: 122, 5: 150}
{2: 104, 4: 130, 5: 140, 6: 111, 7: 127, 1: 116, 8: 142, 3: 130}


Преобразуем в более адекватный вид и разместим в порядке возрастания:

In [23]:
chastota_1 = [129,110,115,122,150,128,122,124]
chastota_2 = [116,104,130,130,140,111,127,142]
print(chastota_1)
print(chastota_2)

[129, 110, 115, 122, 150, 128, 122, 124]
[116, 104, 130, 130, 140, 111, 127, 142]


Воспользуемся формулой для проверки однородности при помощи $X^{2}$:
$$\dot{X}_{n}^{2}=n_{1}n_{2}\sum\limits_{i=1}^{n}\tfrac{(\tfrac{\nu_{1}}{n_{1}}-\tfrac{\nu_{2}}{n_{2}})^{2}}{\nu_{1}+\nu_{2}}$$

In [42]:
sum_xu_kv = 0
for i in range(8):
    sum_xu_kv = sum_xu_kv+(((chastota_1[i]/1000-chastota_2[i]/1000)**2)/(chastota_1[i]+chastota_2[i]))

sum_xu_kv = 1000*1000*sum_xu_kv
print(sum_xu_kv)

4.902835144674852


Найдем значение $t_{\alpha}=X_{1-\alpha}^{2}((k-1)(N-1))$
Воспользуемся таблицей для нахождения. 
Так как $(K-1)(N-1)=7$ и для $\alpha=0,05$, то $t_{\alpha}=14,1$.


In [44]:
talpha=14.1
if sum_xu_kv<talpha:
 print('Гипотеза принимается')
else:
 print('Гипотеза не принимается')

Гипотеза принимается


# Домашнее задание 5
# Различие гипотез

## Дискретное равномерное

### Выбор данных

Возьмем две выборки для дискретного равномерного распределения. Для которых будут параметры 8 и 16.

In [69]:
print('Выборка с параметром 8:',first_sample,'\n','Выборка с параметром 16', second_sample)

Выборка с параметром 8: [6, 8, 4, 1, 6, 7, 6, 4, 4, 8, 1, 8, 3, 8, 8, 1, 3, 3, 4, 8, 3, 2, 1, 7, 2, 8, 2, 4, 5, 7, 6, 4, 5, 4, 7, 8, 7, 8, 8, 4, 4, 1, 1, 7, 3, 5, 3, 6, 2, 2, 7, 2, 3, 3, 5, 6, 3, 8, 7, 3, 6, 8, 6, 7, 5, 6, 4, 3, 6, 2, 6, 5, 5, 8, 7, 2, 5, 1, 5, 1, 6, 4, 2, 7, 4, 2, 1, 5, 2, 5, 4, 6, 8, 4, 2, 4, 2, 6, 7, 4, 7, 6, 4, 4, 4, 3, 4, 5, 2, 4, 6, 5, 7, 6, 4, 5, 4, 3, 7, 4, 5, 3, 4, 1, 1, 3, 1, 6, 3, 8, 8, 1, 7, 7, 4, 2, 6, 3, 4, 5, 6, 3, 3, 4, 3, 7, 3, 6, 3, 8, 7, 1, 3, 5, 3, 1, 7, 4, 3, 2, 4, 3, 2, 1, 4, 7, 5, 2, 5, 7, 3, 3, 4, 6, 5, 5, 2, 2, 4, 2, 1, 7, 3, 2, 5, 5, 6, 5, 1, 5, 8, 3, 4, 5, 5, 5, 5, 5, 3, 2, 3, 7, 8, 1, 3, 6, 4, 6, 5, 4, 5, 3, 1, 5, 4, 4, 7, 1, 2, 1, 2, 5, 7, 7, 8, 7, 5, 6, 8, 1, 2, 2, 3, 2, 6, 6, 7, 2, 4, 6, 4, 3, 5, 8, 2, 8, 7, 6, 5, 6, 5, 5, 5, 8, 1, 6, 6, 8, 2, 5, 2, 3, 4, 3, 7, 1, 3, 6, 2, 4, 4, 1, 3, 1, 7, 2, 7, 1, 1, 8, 5, 2, 2, 8, 2, 1, 1, 5, 8, 5, 1, 8, 2, 5, 1, 4, 2, 2, 3, 2, 1, 6, 2, 7, 2, 2, 1, 5, 5, 3, 2, 1, 6, 4, 2, 5, 2, 4, 1, 5, 1, 6, 5, 8, 3, 

### Постановка задачи

**Что такое $H_{0}, H_{1}$, ошибка первого и второго рода, функция мощности**

Пускай дана выборка $X=(X_{1},...,X_{n})$ из неизвестного совместного распределения $P^{x}$, и поставлена бинарная задача проверки статистических гипотез: $H_{0}, H_{1}$

Здесь $H_{0}$ является нулевой гипотезой 
$H_{1}$ является альтернативной гипотезой. 

Нулевая гипотеза($H_{0}$) - утверждение о параметре генеральной совокупности (параметрах генеральных совокупностей) или распределении, которое необходимо проверить.

Альтернативная гипотеза($H_{1}$) - утверждение, противположное нулевой гипотезе. Выдвигается, но не проверяется.

Тогда пусть $f:R^{n}\to [H_{0},H_{1}]$ статистический критерий.
сопоставляющий каждой реализации выборки $X=x$ одну из имеющихся гипотез. Тогда возможны следующие четыре ситуации:
1. Распределение $P^{x}$ выборки $X$ соответсвует гипотезе $H_{0}$, и она точно определена статистическим критерием, то есть $f(x)=H_{0}$
2. Распределение $P^{x}$ выборки $X$ соответсвует гипотезе $H_{0}$, но она неверно отвергнута статистическим критерием, то есть $f(x)=H_{1}$
3. Распределение $P^{x}$ выборки $X$ соответсвует гипотезе $H_{1}$, и она точно определена статистическим критерием, то есть $f(x)=H_{1}$
4. Распределение $P^{x}$ выборки $X$ соответсвует гипотезе $H_{1}$, но она неверно отвергнута статистическим критерием, то есть $f(x)=H_{0}$ 

Во втором и четвертом случае говорят, что произошла статистическая ошибка, и ее называют ошибкой первого и второго рода соответственно.

Ошибка первого рода - вероятность отвергнуть верную нулевую гипотезу
Ошибка второго рода - веротяность не отвергнуть невенрную нулевую гипотезу

Функция $l(x)=\tfrac{L_{1}(x)}{L_{0}(x)}$ определенная для {x:$L_{1}(x)+L_{2}(x)>0$}, называется функцией отношения правдоподобия.


In [57]:
display(Image(url='https://present5.com/presentation/3/55256088_179121202.pdf-img/55256088_179121202.pdf-9.jpg') )

$P(X\in X_{1}|H_{0})=\alpha$ - ошибка первого рода

$P(X\in X_{2}|H_{1})=\beta$ - ошибка второго рода

### Вычисление функции отношения правдоподобия

Функция $l(x)=\tfrac{L_{1}(x)}{L_{0}(x)}$ определенная для {x:$L_{1}(x)+L_{2}(x)>0$}, называется функцией отношения правдоподобия.

Найдем функции правдоподобия для наших выборок. 
$L_{0}(x)=f_{R[1,8]}(x_{1})+...+f_{R[1,8]}(x_{n})=\tfrac{1}{8}I(0 \leq x_{1} \leq 8) ... \tfrac{1}{8}I(0 \leq x_{n} \leq 8)=\tfrac{1}{8^{n}} I(x \in [1,8]^{n})$

$L_{1}(x)=f_{R[1,16]}(x_{1})+...+f_{R[1,16]}(x_{n})=\tfrac{1}{16}I(0 \leq x_{1} \leq 16) ... \tfrac{1}{16}I(0 \leq x_{n} \leq 16)=\tfrac{1}{16^{n}} I( x \in [1,16]^{n})$

$l(x)=\tfrac{L_{1}(x)}{L_{0}(x)}=\tfrac{\tfrac{1}{16^{n}} I( x \in [1,16]^{n})}{\tfrac{1}{8^{n}} I(x \in [1,8]^{n})}=\tfrac{\tfrac{1}{2^{n}}\cdot \tfrac{1}{8^{n}}I(x\in[1,16]^{n})}{\tfrac{1}{8^{n}}I(x\in[1,8]^{n})}=\tfrac{1}{2^{n}}\tfrac{I(x\in[1,16]^{n})}{I(x\in[1,8]^{n})}$

In [77]:
a = len(first_sample)
b = len(second_sample)

ind_1 = 1
ind_2 = 1

for i in range(a):
    if first_sample[i] >= 1 and first_sample[i] <= 8:
        ind_1 = ind_1*1/8
    else: 
        ind_1 = ind_1*0
for i in range(b):
    if second_sample[i] >= 1 and second_sample[i] <= 16:
        ind_2 = ind_2*1/16
    else: 
        ind_2 = ind_2*0   

Посчитаем функцию отношения правдоподобия:

In [78]:
lx = (ind_2/ind_1)*(1/2)**1000
print(lx)

ZeroDivisionError: float division by zero

У нас значение ind_1 и ind_2 будут очень маленькими, так как мы много раз умножаем их. тогда будем ими принебригать.

Тогда функция отношения правдоподобия:

In [79]:
lx = (1/(2**1000))
print(lx)

9.332636185032189e-302


### Вычисление критической области

Функция $\pi(x)$ называется критической функцикй. Соответсвуюдий критической функции $\pi(x)$ критерий также обозначим буквой $\pi$

Функция мощности критерия $\pi$ определяется равенством
$$W(\pi)=E\pi(X)=1-\beta(\pi)$$

$$\alpha(\pi)=E\pi(x)\text{вероятность ошибки 1-ого рода}$$
$$\beta(\pi)=1-E\pi(x)\text{вероятность ошибки 2-ого рода}$$

Если $\alpha(\pi)\leq \varepsilon$, то говорят, что критерий $\pi$ имеет уровень значимости $\varepsilon$
Тогда наиболее мощный критерий проверки двух простых гипотез на уровне значимости $\varepsilon$ называется рандомизированный критерий $\pi$, который является решением оптимизационной задачи
$$W(\pi)\to max, \alpha(\pi)=\varepsilon$$
Решение этой оптимизационной задачи не обязано существовать для любых $\varepsilon$, а в случае существование не обязано быть единственным. 

В случае двух гипотез $H_{0}, H_{1}$ множество называют критической областью гипотезы $H_{0}$. Критическую область обычно задают с помощью вспомогательной статистики- меры отколонения эмпирических данных от гипотетических. Тогда критическая область- это просто область достаточно больших значений вспомогательной статистики. Это вспомогательну. статистику называют статистикой критерия.

Критическая область - это область отвержения нулевой гипотезы.

Вычислить критическую область можно с помощью формулы $W(\pi)=E\pi(X)=1-\beta(\pi)$. Сделаем некоторые преобразования.
$$W=1-\tfrac{1-\alpha}{2^{n}}$$
$$\beta=1-W=\tfrac{1-\alpha}{2^{n}}$$
Тогда с помощью кода на Python вычислим эти значения:

In [1]:
alpha = 0.05
W = 1 - ((1-alpha)/(2**1000))
print(W)

1.0


In [3]:
beta = 1 - W
print(beta)

0.0


### Вычисление минимального необходимого количества материала при фиксации минимального возможного значения ошибок первого и второго рода.